In [ ]:
import datetime
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import rioxarray

import matplotlib.pyplot as plt


In [ ]:
SEVERN_EASTING_RANGE = [280000, 400000]
SEVERN_NORTHING_RANGE = [260000, 360000]

SEVERN_ID_TO_NAMES = {"Abermule": 54014, "Dolwen": 54080, "Plynlimon Flume": 54022,\
                      "Bewdley": 54001}

In [ ]:
bewdley_shp = gpd.read_file('Catchment_Data/Bewdley/54001/54001.shp')

abermule_shp = gpd.read_file('Catchment_Data/Abermule/54014/54014.shp')
dolwen_shp = gpd.read_file('Catchment_Data/Dolwen/54080/54080.shp')
plynlimon_shp = gpd.read_file('Catchment_Data/Plynlimon Flume/54022/54022.shp')

In [ ]:
severn_hght = rioxarray.open_rasterio('Catchment_Data/HGHT_SEVERN_1km.tif')
severn_hght = severn_hght.sortby('y')
severn_hght = severn_hght.sel(band=1)

In [ ]:
def make_region_hght_clip(region_shp, hght_data):
    region_clip = hght_data.rio.clip(region_shp.geometry.values, region_shp.crs, drop=False, invert=False)
    return region_clip

In [ ]:
def coerse_data_into_haduk_format(one_day_data, offset, xrange, yrange):
    """
    Quick fix for coersing data to have same grid as HADUK.
    """
    one_day_data = one_day_data.assign_coords(x=(one_day_data['x'] + offset))
    one_day_data = one_day_data.assign_coords(y=(one_day_data['y'] + offset))
    one_day_data = one_day_data.sel(x=xrange, y=yrange)
    one_day_data = one_day_data.rename({'x': 'projection_x_coordinate', 'y': 'projection_y_coordinate'})
    return one_day_data


In [ ]:
def make_region_mask(region_data):
    """
    Make binary mask of clipped region data data
    """
    return (region_data/region_data.where(region_data>0))